# Simple Dataset Demo
Embedded audio in parquet. Splits: lt1m, lt5m, lt30m. *Generated by Claude*

In [ ]:
from oyez_sa_asr.loaders import load_simple, play_audio

# Load short utterances (<1 min)
ds = load_simple("lt1m")
print(f"lt1m utterances: {len(ds):,}")

In [ ]:
# Play first utterance
sample = ds[0]
print(f"Speaker: {sample['speaker']}")
print(f"Text: {sample['sentence']}")
play_audio(sample["audio"]["bytes"])

In [ ]:
# Load all splits
all_ds = load_simple("all")
by_split = {}
for u in all_ds:
    by_split[u["_split"]] = by_split.get(u["_split"], 0) + 1
print("Utterances by split:", by_split)